# 2022 Data Collection

## Introduction
#### The ***2022* Formula 1 season** marked the beginning of the ground-effect era under new technical regulations and was defined by a dominant **Max Verstappen**, who secured his **second World Championship** early as Red Bull mastered the regulations, while **Ferrari** emerged as the early-season challenger with a fast but unreliable car that faded due to strategic errors and reliability issues; the season featured **22 races**, the introduction of sprint weekends in expanded form, dramatic regulation-driven performance swings across the grid, and a reshuffled competitive order where Red Bull pulled away decisively, Ferrari stumbled, and Mercedes struggled before recovering late, making 2022 a season of **technical upheaval**, **strategic missteps**, and **regulation-driven dominance**.


## Importing the necessary libraries

In [1]:
import pandas as pd
import fastf1
import logging

In [2]:
fastf1.Cache.enable_cache("../cache")

## Getting the Races of 2022

In [3]:
schedule = fastf1.get_event_schedule(2022)
listOfRaces = schedule["Country"].tolist()
listOfRaces

['Spain',
 'Bahrain',
 'Bahrain',
 'Saudi Arabia',
 'Australia',
 'Italy',
 'United States',
 'Spain',
 'Monaco',
 'Azerbaijan',
 'Canada',
 'Great Britain',
 'Austria',
 'France',
 'Hungary',
 'Belgium',
 'Netherlands',
 'Italy',
 'Singapore',
 'Japan',
 'United States',
 'Mexico',
 'Brazil',
 'Abu Dhabi']

## Iterating into every Race and concatenating every Race Data into a DataFrame
Through my first try in ["Bahrain_test.ipynb"](https://github.com/Chracker24/MTS-IE/blob/main/02_Notebooks/Data_Collection/Formula1/2020/Bahrain_test.ipynb), I am going to use a loop to collect data and trim it down to the column and data points that I deem necessary for the Intelligence Engine.

In [4]:
cols = [
    "Driver",
    "LapTime",
    "LapNumber",
    "Stint",
    "Sector1Time",
    "Sector2Time",
    "Sector3Time",
    "Compound",
    "Team",
    "Deleted",
]

In [5]:
Season_Data2022 = []

In [10]:
logging.getLogger("fastf1").setLevel(logging.ERROR)
for i in range(len(listOfRaces)):
    session = fastf1.get_session(2022, listOfRaces[i], "R")
    session.load(laps=True, telemetry=False, weather=False)
    laps = session.laps
    Race_Data = laps[cols].copy()
    Race_Data = Race_Data.dropna(subset=["LapTime"]).copy()
    Race_Data["Season"] = 2022
    Race_Data["Race"] = listOfRaces[i]
    Race_Data[["LapTime","Sector1Time","Sector2Time","Sector3Time"]] = Race_Data[["LapTime","Sector1Time","Sector2Time","Sector3Time"]].apply(lambda x : x.dt.total_seconds())
    Race_Data = Race_Data[Race_Data["Deleted"]==False].copy()

    Season_Data2022.append(Race_Data)

In [11]:
season_2022 = pd.concat(Season_Data2022, ignore_index=True)

In [14]:
season_2022 = season_2022.sort_values("LapNumber").reset_index(drop=True)
season_2022

,Driver,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,Compound,Team,Deleted,Season,Race
0,OCO,93.887,1.0,1.0,NaN,32.554,25.027,MEDIUM,Alpine,False,2022,Hungary
1,RIC,101.889,1.0,1.0,NaN,33.375,30.519,SOFT,McLaren,False,2022,Hungary
2,LEC,89.695,1.0,1.0,NaN,18.645,36.781,MEDIUM,Ferrari,False,2022,Australia
3,MAG,101.200,1.0,1.0,NaN,40.993,35.645,HARD,Haas F1 Team,False,2022,Abu Dhabi
4,VER,92.198,1.0,1.0,NaN,38.449,33.249,MEDIUM,Red Bull Racing,False,2022,Abu Dhabi
...,...,...,...,...,...,...,...,...,...,...,...,...
74491,MAG,76.825,72.0,5.0,26.393,27.338,23.094,SOFT,Haas F1 Team,False,2022,Netherlands
74492,NOR,75.351,72.0,4.0,25.710,27.009,22.632,SOFT,McLaren,False,2022,Netherlands
74493,VET,76.150,72.0,5.0,26.029,27.107,23.014,SOFT,Aston Martin,False,2022,Netherlands
74494,VER,75.061,72.0,5.0,25.693,26.339,23.029,SOFT,Red Bull Racing,False,2022,Netherlands


## Adding the Dataframe as a CSV file in Data/Archive

In [ ]:
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[3]
archive_dir = PROJECT_ROOT / "Data" / "Archive"

archive_dir.mkdir(parents=True, exist_ok=True)

season_2022.to_csv(
    archive_dir / "season_2022.csv",
    index=False
)